# SVM 與 MLP 的訓練過程、最佳化推導與程式實作（含 w^{*} = w + Δw）

本筆記包含：
1. **SVM 的訓練過程**：Soft-Margin（合頁損失）定義、次梯度、SGD 更新、（附）dual 觀念。
2. **MLP 的訓練過程**：前向傳播、交叉熵損失、反向傳播（Backprop）梯度推導。
3. **最佳的 W 推導直觀**：以 ∇L=0 與一階泰勒展開說明為何選擇 ΔW = -η ∇L。
4. **在程式中實現** w^{*} = w + Δw：提供 NumPy 與 PyTorch（概念）對照與可跑的最小訓練範例（含 Loss 圖）。

## 1. SVM 的訓練過程（線性、Soft-Margin、二分類）

模型與符號：資料 (x_i, y_i)，其中 x_i∈R^d，y_i∈{-1,+1}；分類面 f(x)=w^T x + b。

Primal（標準軟間隔）：

$$
\min_{w,b,\xi} \; \tfrac{1}{2}\|w\|^2 + C \sum_{i=1}^n \xi_i \quad \text{s.t. } y_i(w^\top x_i + b) \ge 1 - \xi_i,\; \xi_i \ge 0.
$$

實作常用的等價形式（合頁損失 + L2）：

$$
L(w,b)=\tfrac{\lambda}{2}\|w\|^2 + \tfrac{1}{n}\sum_{i=1}^{n}\max(0,\,1-y_i(w^\top x_i+b)).
$$

其中 λ ≈ 1/(Cn)。

次梯度（以單一樣本 (x_i,y_i)）：令 m_i = y_i(w^T x_i+b)。

- 若 m_i ≥ 1： ∂_w L = λ w，∂_b L = 0  
- 若 m_i < 1： ∂_w L = λ w − y_i x_i，∂_b L = −y_i

SGD 更新（即 w^{*} = w + Δw）：

- 若 m_i ≥ 1： Δw = −η λ w，Δb = 0  
- 若 m_i < 1： Δw = −η λ w + η y_i x_i，Δb = η y_i  

更新則為 w ← w + Δw,  b ← b + Δb。

（附）Dual 觀念：以拉格朗日乘子 α_i 可得對偶：

$$
\max_\alpha \sum_{i=1}^n \alpha_i - \tfrac{1}{2}\sum_{i,j} \alpha_i\alpha_j y_i y_j (x_i^\top x_j)
\quad \text{s.t. } 0\le \alpha_i \le C,\; \sum_i \alpha_i y_i = 0.
$$

最優時 w^{*}=\sum_i \alpha_i y_i x_i；b 可由 KKT 條件求得。

In [ ]:
# SVM：Hinge + L2，簡易 SGD 訓練示範（NumPy）
import numpy as np
import matplotlib.pyplot as plt

def make_toy_linear_separable(n_per_class=100, d=2, seed=0):
    rng = np.random.default_rng(seed)
    mean_pos = np.ones(d)
    mean_neg = -np.ones(d)
    cov = np.eye(d) * 0.3
    X_pos = rng.multivariate_normal(mean_pos, cov, size=n_per_class)
    X_neg = rng.multivariate_normal(mean_neg, cov, size=n_per_class)
    X = np.vstack([X_pos, X_neg])
    y = np.hstack([np.ones(n_per_class), -np.ones(n_per_class)])
    return X, y

def svm_hinge_loss(w, b, X, y, lam):
    margins = y * (X @ w + b)
    hinge = np.maximum(0.0, 1.0 - margins)
    return 0.5 * lam * (w @ w) + hinge.mean()

def svm_sgd_train(X, y, epochs=50, eta=0.1, lam=1e-2, seed=0):
    rng = np.random.default_rng(seed)
    n, d = X.shape
    w = rng.normal(scale=0.1, size=d)
    b = 0.0
    losses = []
    for ep in range(epochs):
        idx = rng.permutation(n)
        for i in idx:
            xi, yi = X[i], y[i]
            margin = yi * (xi @ w + b)
            if margin >= 1.0:
                delta_w = -eta * lam * w
                delta_b = 0.0
            else:
                delta_w = -eta * lam * w + eta * yi * xi
                delta_b = eta * yi
            w = w + delta_w
            b = b + delta_b
        losses.append(svm_hinge_loss(w, b, X, y, lam))
    return w, b, losses

X, y = make_toy_linear_separable(n_per_class=120, d=2, seed=42)
w, b, losses = svm_sgd_train(X, y, epochs=60, eta=0.1, lam=1e-2, seed=1)
print('final loss:', float(losses[-1]))
pred = np.sign(X @ w + b)
acc = (pred == y).mean()
print('train acc:', float(acc))

plt.figure()
plt.plot(losses)
plt.title('SVM training loss')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()


## 2. MLP（1 隱藏層）訓練過程與 Backprop 推導

前向傳播（輸入 x∈R^d、隱藏 h、類別 C）：

$$
\begin{aligned}
a^{(1)} &= W_1 x + b_1,\quad h=\sigma(a^{(1)}) \\
z &= W_2 h + b_2,\quad \hat{y}=\operatorname{softmax}(z)
\end{aligned}
$$

交叉熵損失（多類）：

$$
L = -\frac{1}{n}\sum_{i=1}^{n}\sum_{k=1}^{C} y_{ik}\,\log \hat{y}_{ik}.
$$

反向傳播（softmax + CE）：令 δ^{(2)}=ŷ−y。

$$
\nabla_{W_2}L=\delta^{(2)}h^\top,\quad \nabla_{b_2}L=\sum \delta^{(2)}.
$$
$$
\delta^{(1)}=(W_2^\top \delta^{(2)})\odot \sigma'(a^{(1)}),\quad
\nabla_{W_1}L=\delta^{(1)}x^\top,\quad \nabla_{b_1}L=\sum \delta^{(1)}.
$$

帶 L2 正則：在各 ∇_{W_ℓ}L 上再加 λ W_ℓ。

參數更新：W_ℓ ← W_ℓ − η ∇_{W_ℓ}L,  b_ℓ ← b_ℓ − η ∇_{b_ℓ}L，即 w^{*}=w+Δw 且 Δw=−η∇L。

In [ ]:
# MLP（softmax + CE）訓練示範（NumPy）
import numpy as np
import matplotlib.pyplot as plt

def softmax(Z):
    Z = Z - Z.max(axis=0, keepdims=True)
    E = np.exp(Z)
    return E / E.sum(axis=0, keepdims=True)

def relu(A):
    return np.maximum(0.0, A)

def relu_grad(A):
    return (A > 0).astype(A.dtype)

def cross_entropy(probs, Y):
    eps = 1e-12
    return -np.mean(np.sum(Y * np.log(probs + eps), axis=0))

def make_blobs_2class(n_per_class=150, d=2, seed=0):
    rng = np.random.default_rng(seed)
    mean1 = np.array([1.2, 0.8])
    mean2 = np.array([-1.1, -0.7])
    cov = np.eye(2) * 0.7
    X1 = rng.multivariate_normal(mean1, cov, size=n_per_class)
    X2 = rng.multivariate_normal(mean2, cov, size=n_per_class)
    X = np.vstack([X1, X2])
    y = np.hstack([np.zeros(n_per_class, dtype=int), np.ones(n_per_class, dtype=int)])
    return X, y

def to_onehot(y, C):
    Y = np.zeros((C, y.size))
    Y[y, np.arange(y.size)] = 1.0
    return Y

def mlp_train(X, y, h=16, epochs=100, eta=0.1, lam=1e-3, seed=0):
    rng = np.random.default_rng(seed)
    d = X.shape[1]
    C = int(y.max()) + 1
    Xb = X.T
    Y = to_onehot(y, C)
    B = Xb.shape[1]

    W1 = rng.normal(scale=0.1, size=(h, d)); b1 = np.zeros((h, 1))
    W2 = rng.normal(scale=0.1, size=(C, h)); b2 = np.zeros((C, 1))

    losses = []
    for ep in range(epochs):
        A1 = W1 @ Xb + b1
        H = relu(A1)
        Z = W2 @ H + b2
        Yhat = softmax(Z)
        loss = cross_entropy(Yhat, Y) + 0.5 * lam * (np.sum(W1*W1) + np.sum(W2*W2))
        losses.append(loss)

        delta2 = (Yhat - Y) / B
        dW2 = delta2 @ H.T + lam * W2
        db2 = delta2.sum(axis=1, keepdims=True)

        delta1 = (W2.T @ delta2) * relu_grad(A1)
        dW1 = delta1 @ Xb.T + lam * W1
        db1 = delta1.sum(axis=1, keepdims=True)

        W2 = W2 - eta * dW2
        b2 = b2 - eta * db2
        W1 = W1 - eta * dW1
        b1 = b1 - eta * db1

    A1 = W1 @ Xb + b1
    H = relu(A1)
    Z = W2 @ H + b2
    Yhat = softmax(Z)
    y_pred = np.argmax(Yhat, axis=0)
    acc = (y_pred == y).mean()
    return (W1, b1, W2, b2), losses, acc

X, y = make_blobs_2class(n_per_class=160, d=2, seed=7)
params, losses, acc = mlp_train(X, y, h=16, epochs=120, eta=0.15, lam=1e-3, seed=3)
print('final loss:', float(losses[-1]))
print('train acc:', float(acc))

plt.figure()
plt.plot(losses)
plt.title('MLP training loss')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()


## 3. 「最佳 W」的數學推導（直觀版）

必要條件：可微時，極值點滿足 ∇_W L(W^{*})=0；凸問題（如 hinge+L2 的 w 部分）亦為全域最優條件。

為何用 ΔW=−η ∇L？（一階泰勒）：

$$
L(W+ΔW) ≈ L(W) + (∇L(W))^T ΔW.
$$

沿 −∇L 方向取小步長 η>0，即 ΔW=−η ∇L(W)，可使近似下降。

SVM：Dual 得 w^{*}=∑ α_i y_i x_i 並由 KKT 求 b；或在 Primal（合頁損失）上做次梯度下降到收斂。  
MLP：非凸，以反向傳播計算 ∇L 並用梯度法（SGD/Adam 等）迭代尋找良好解。

## 4. 在程式中實現 w^{*}=w+Δw（快速對照）

**NumPy（已知梯度）**：

```python
w = w + (-eta * grad_w)  # Δw = -η ∇L
```

**在 SVM 更新裡**（對應前述分支）：

```python
if margin >= 1:
    delta_w = -eta * lam * w
else:
    delta_w = -eta * lam * w + eta * y * x
w = w + delta_w
```

**PyTorch（概念示意）**：

```python
loss.backward()
with torch.no_grad():
    for p in model.parameters():
        p += -eta * p.grad   # w* = w + Δw
        p.grad.zero_()
```


In [ ]:
# 額外示範：顯式計算 Δw 並更新
import numpy as np

w = np.array([1.0, -2.0, 0.5])
grad_w = np.array([0.3, -0.2, 0.1])
eta = 0.05
delta_w = -eta * grad_w
w_new = w + delta_w
print('w:', w)
print('grad_w:', grad_w)
print('delta_w:', delta_w)
print('w_new (w*):', w_new)
